In [21]:
import matplotlib.pyplot as plt
%matplotlib inline

In [22]:
# Setting up the Environment.
!pip install pandas numpy matplotlib seaborn yfinance scipy statsmodels scikit-learn

# Installing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from scipy.stats import pearsonr
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import StandardScaler
%matplotlib notebook

# Fetching Financial Data from Yahoo Finance
# Retrieving Meta Inc.'s historical stock price and financial statements.

# Defining the company ticker
meta_ticker = "META"

# Download stock price data (last 5 years)
meta_stock = yf.download(meta_ticker, start="2021-01-01", end="2025-01-01")

# Print the first few rows and info of meta_stock immediately after download
print("Shape of meta_stock after download:", meta_stock.shape)
print("\nFirst few rows of meta_stock after download:\n", meta_stock.head())
print("\nInformation about meta_stock after download:\n", meta_stock.info())

# Download financial statements
meta_info = yf.Ticker(meta_ticker)
meta_balance_sheet = meta_info.balance_sheet
meta_income_statement = meta_info.financials
meta_cashflow = meta_info.cashflow

# Data Preprocessing
# Clean and structure the stock price data and financial statements.

# Transpose financial statements for potential readability (we might use the original later for calculations)
balance_sheet_transposed = meta_balance_sheet.T / 1e9
income_statement_transposed = meta_income_statement.T / 1e9
cashflow_statement_transposed = meta_cashflow.T / 1e9

# Display sample cleaned data
print("\nSample Balance Sheet (Transposed):\n", balance_sheet_transposed.head())
print("\nSample Income Statement (Transposed):\n", income_statement_transposed.head())
print("\nSample Cash Flow Statement (Transposed):\n", cashflow_statement_transposed.head())

# Exploratory Data Analysis (EDA)
# Visualize trends in stock price movements and revenue growth.

# Stock Price Analysis
plt.figure(figsize=(12, 6))
plt.plot(meta_stock['Close'], color='blue', label="Meta Stock Price (Close)")
plt.title("Meta Inc. Stock Price (Jan 2021 - Jan 2025)")
plt.xlabel("Date")
plt.ylabel("Stock Price (USD)")
plt.legend()
plt.grid(True)
plt.show()

# Revenue Growth Visualization and Calculation
plt.figure(figsize=(12, 6))
sns.barplot(x=meta_income_statement.columns, y=meta_income_statement.loc["Total Revenue"], color='green')
plt.title("Meta Inc. Revenue Over Time")
plt.xlabel("Year")
plt.ylabel("Revenue (Billion USD)")
plt.grid(axis='y')
plt.show()

# Calculate Year-over-Year Revenue Growth
revenue_growth = meta_income_statement.loc["Total Revenue"].pct_change() * 100
print("\nYear-over-Year Revenue Growth (%):\n", revenue_growth)

# Financial Ratio Calculation
# Calculate and analyze key financial health indicators.

# Calculate Profit Margin
profit_margin = meta_income_statement.loc["Net Income"] / meta_income_statement.loc["Total Revenue"]

# Debt-to-Equity Ratio
debt_equity_ratio = meta_balance_sheet.loc["Total Liabilities Net Minority Interest"] / meta_balance_sheet.loc["Stockholders Equity"]

# Return on Equity (ROE)
roe = meta_income_statement.loc["Net Income"] / meta_balance_sheet.loc["Stockholders Equity"]

# Print financial ratios
financial_ratios = pd.DataFrame({
    "Profit Margin": profit_margin,
    "Debt-to-Equity Ratio": debt_equity_ratio,
    "Return on Equity": roe
})
print("\nFinancial Ratios:\n", financial_ratios)

# Time-Series Forecasting
# Predict future stock prices using an ARIMA model.

# Prepare dataset for modeling
if not meta_stock.empty:
    # Calculate returns first (handling potential NaNs from the start)
    meta_stock['Returns'] = meta_stock['Close'].pct_change()
    meta_stock.dropna(inplace=True)

    meta_stock.index = pd.to_datetime(meta_stock.index)

    # Interpolate missing trading days *before* setting daily frequency
    meta_stock_filled = meta_stock.resample('D').interpolate(method='linear')

    try:
        model = ARIMA(meta_stock_filled['Close'], order=(5,1,0))
        meta_forecast = model.fit()
        print("\nARIMA model fitted successfully!") # Added print statement

        forecast_prices = meta_forecast.forecast(steps=30)

        plt.figure(figsize=(12, 6))
        plt.plot(meta_stock_filled['Close'], label="Actual Stock Price", color="blue")
        forecast_dates = pd.date_range(start=meta_stock_filled.index[-1] + pd.Timedelta(days=1), periods=30, freq="D")
        plt.plot(forecast_dates, forecast_prices, label="Forecasted Price", color="red")
        plt.title("Meta Inc. Stock Price Prediction (Next 30 Days)")
        plt.xlabel("Date")
        plt.ylabel("Stock Price (USD)")
        plt.legend()
        plt.grid(True)
        plt.savefig("meta_stock_forecast.png")
        plt.show()
        print("\nForecast plot should be displayed now.")
    except Exception as e:
        print(f"\nError during ARIMA forecasting: {e}")
else:
    print("\nError: Stock price data is empty. Cannot perform ARIMA forecasting.")

# Troubleshooting
if not meta_stock.empty:
    print("\nMissing values in interpolated stock data:\n", meta_stock_filled.isnull().sum())
    print("\nFirst few rows of interpolated stock data:\n", meta_stock_filled.head())
    print("\nInformation about interpolated stock data:\n", meta_stock_filled.info())
else:
    print("\nStock data is empty. Please check the data fetching step.")

[*********************100%***********************]  1 of 1 completed

Shape of meta_stock after download: (1005, 5)

First few rows of meta_stock after download:
 Price            Close        High         Low        Open    Volume
Ticker            META        META        META        META      META
Date                                                                
2021-01-04  267.678436  273.710007  263.955990  273.491038  15106100
2021-01-05  269.698914  271.122198  266.951851  267.031492   9871600
2021-01-06  262.074860  266.494035  258.790351  260.771007  24354100
2021-01-07  267.479340  270.335872  263.537924  264.652666  15789800
2021-01-08  266.314911  267.688443  261.945489  267.051430  18528300
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1005 entries, 2021-01-04 to 2024-12-31
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   (Close, META)   1005 non-null   float64
 1   (High, META)    1005 non-null   float64
 2   (Low, META)     1005 non-null   float64
 3   (


Sample Balance Sheet (Transposed):
            Treasury Shares Number Ordinary Shares Number Share Issued  \
2024-12-31                    NaN               2.534488     2.534488   
2023-12-31                    0.0                  2.561        2.561   
2022-12-31                    NaN                  2.614        2.614   
2021-12-31                    NaN                  2.741        2.741   
2020-12-31                    NaN                    NaN          NaN   

           Total Debt Tangible Book Value Invested Capital Working Capital  \
2024-12-31      49.06             161.068          211.463          66.449   
2023-12-31     37.234             131.726          171.553          53.405   
2022-12-31     26.591              104.51          135.636          32.523   
2021-12-31     13.873             105.048          124.879          45.531   
2020-12-31        NaN                 NaN              NaN             NaN   

           Net Tangible Assets Capital Lease Obligation

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

C:\Users\Naomie  Fillies\AppData\Local\Temp\ipykernel_11976\3857501760.py:71: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  revenue_growth = meta_income_statement.loc["Total Revenue"].pct_change() * 100
C:\Users\Naomie  Fillies\AppData\Local\Temp\ipykernel_11976\3857501760.py:71: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  revenue_growth = meta_income_statement.loc["Total Revenue"].pct_change() * 100



Year-over-Year Revenue Growth (%):
 2024-12-31          NaN
2023-12-31   -17.993204
2022-12-31   -13.560214
2021-12-31     1.131988
2020-12-31     0.000000
Name: Total Revenue, dtype: float64

Financial Ratios:
            Profit Margin Debt-to-Equity Ratio Return on Equity
2024-12-31      0.379086              0.51149         0.341442
2023-12-31      0.289825             0.499158         0.255262
2022-12-31      0.198955             0.477389         0.184547
2021-12-31      0.333845             0.329183         0.315265
2020-12-31           NaN                  NaN              NaN

ARIMA model fitted successfully!


<IPython.core.display.Javascript object>


Forecast plot should be displayed now.

Missing values in interpolated stock data:
 Price    Ticker
Close    META      0
High     META      0
Low      META      0
Open     META      0
Volume   META      0
Returns            0
dtype: int64

First few rows of interpolated stock data:
 Price            Close        High         Low        Open        Volume  \
Ticker            META        META        META        META          META   
Date                                                                       
2021-01-05  269.698914  271.122198  266.951851  267.031492  9.871600e+06   
2021-01-06  262.074860  266.494035  258.790351  260.771007  2.435410e+07   
2021-01-07  267.479340  270.335872  263.537924  264.652666  1.578980e+07   
2021-01-08  266.314911  267.688443  261.945489  267.051430  1.852830e+07   
2021-01-09  262.755000  265.870322  259.530187  264.453659  2.248963e+07   

Price        Returns  
Ticker                
Date                  
2021-01-05  0.007548  
2021-01-06 -0.

In [25]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Time-Series Forecasting and Evaluation
# Predict the stock price of the last 30 days and evaluate accuracy.

if not meta_stock.empty:
    meta_stock.index = pd.to_datetime(meta_stock.index)
    meta_stock_filled = meta_stock.resample('D').interpolate(method='linear')
    meta_stock_filled.dropna(subset=[('Close', 'META')], inplace=True) # Correctly refer to the 'Close' column

    # Split data into training and testing sets (last 30 days for testing)
    train_data = meta_stock_filled[:-30][('Close', 'META')]
    test_data = meta_stock_filled[-30:][('Close', 'META')]

    try:
        model = ARIMA(train_data, order=(5,1,0))
        model_fit = model.fit()
        print("\nARIMA model fitted successfully for evaluation!")

        # Make predictions for the test set
        predictions = model_fit.forecast(steps=30)

        # Evaluate the model
        mse = mean_squared_error(test_data, predictions)
        rmse = sqrt(mse)
        print(f'\nMean Squared Error (MSE): {mse:.2f}')
        print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')

        # Plot the actual vs. predicted prices for the last 30 days
        plt.figure(figsize=(12, 6))
        plt.plot(test_data.index, test_data.values, label='Actual Price (Last 30 Days)', color='blue')
        plt.plot(test_data.index, predictions, label='Predicted Price (Last 30 Days)', color='red')
        plt.title('Actual vs. Predicted Stock Price (Last 30 Days)')
        plt.xlabel('Date')
        plt.ylabel('Stock Price (USD)')
        plt.legend()
        plt.grid(True)
        plt.savefig("meta_stock_last_30_days_forecast.png")
        plt.show()

    except Exception as e:
        print(f"\nError during ARIMA forecasting/evaluation: {e}")
else:
    print("\nError: Stock price data is empty. Cannot perform ARIMA forecasting/evaluation.")


ARIMA model fitted successfully for evaluation!

Mean Squared Error (MSE): 661.13
Root Mean Squared Error (RMSE): 25.71


<IPython.core.display.Javascript object>